In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [335]:
from pathlib import Path
import polars as pl
import re
import ezregex as er
import pandas as pd
import concurrent.futures
from jsonpath_ng.ext import *
from unityparser import UnityDocument as UD
from pyparsing import *
from builtins import filter
import numpy as np
from clipboard import copy
import itertools as it
import multiprocessing
import string
import json
from polars import col as c
from tkinter import Tk
from tkinter.filedialog import askdirectory
from utils import *
from load_data import *

In [3]:
# Get the installation path from the user
LANG = 'en-us'
# Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
# INSTALLATION_PATH = Path(askdirectory(mustexist=True, title='Select the KSP root directory'))
INSTALLATION_PATH = Path('/home/zeke/Software/KSP_linux')

In [336]:
from default_config import default_config
PATH_GAMEDATA_OFFSET = len(str(INSTALLATION_PATH / 'GameData'))
part_db_path, part_dump_file, loc_dump_file, localization_paths, thumbs_paths = construct_paths(INSTALLATION_PATH, LANG)
fields, calculated, localized, types, default_fields, column_order, natively_list = construct_config(pd.DataFrame(default_config, columns=['columns', 'path', 'localized', 'type', 'default']))
paths = get_paths(part_db_path)

In [6]:
string_lookup = load_localization(localization_paths)

Failed to parse /home/zeke/Software/KSP_linux/GameData/PlanetaryBaseInc/Localization/en-us.cfg: , found 'en'  (at char 23), (line:3, col:9)


In [7]:
results = load_parts(paths)
failed = []
all_parts = []
for i in results:
    if type(i) is str:
        failed.append(i)
    else:
        all_parts += i
print(f"{len(all_parts)}/{len(all_parts)+len(failed)} parts loaded. {len(failed)} parts failed to parse")

Error parsing /home/zeke/Software/KSP_linux/GameData/PlanetaryBaseInc/ModSupport/Parts/Extraplanetary Launchpads/Workshop.cfg:
, found '|'  (at char 20), (line:1, col:21)
Error parsing /home/zeke/Software/KSP_linux/GameData/MechJeb2/Parts/MechJeb2_AR202/part.cfg:
, found 'MODULE'  (at char 1035), (line:59, col:5)
Error parsing /home/zeke/Software/KSP_linux/GameData/Kethane/Parts/kethane_1m_converter/part.cfg:
, found 'MODULE'  (at char 840), (line:51, col:1)
Error parsing /home/zeke/Software/KSP_linux/GameData/MechJeb2/Parts/MechJeb2_Pod/part.cfg:
Expected end of text, found 'PART'  (at char 151), (line:10, col:1)Error parsing /home/zeke/Software/KSP_linux/GameData/Kethane/Parts/kethane_2m_converter/part.cfg:
, found 'MODULE'  (at char 1195), (line:55, col:1)

Error parsing /home/zeke/Software/KSP_linux/GameData/MechJeb2/Parts/MechJeb2_AR202/part.cfg:
, found 'MODULE'  (at char 1035), (line:59, col:5)
Error parsing /home/zeke/Software/KSP_linux/GameData/PlanetaryBaseInc/ModSupport/Part

In [8]:
thumbs = load_thumbs(thumbs_paths, PATH_GAMEDATA_OFFSET)

In [396]:
from default_config import default_config
PATH_GAMEDATA_OFFSET = len(str(INSTALLATION_PATH / 'GameData'))
part_db_path, part_dump_file, loc_dump_file, localization_paths, thumbs_paths = construct_paths(INSTALLATION_PATH, LANG)
fields, calculated, localized, types, default_fields, column_order, natively_list = construct_config(pd.DataFrame(default_config, columns=['columns', 'path', 'localized', 'type', 'default']))
paths = get_paths(part_db_path)

df = construct_df(fields, all_parts, localized, string_lookup, types, calculated, natively_list, thumbs, PATH_GAMEDATA_OFFSET)
df

title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,experimentActionName,…,heatProduction,exhaustDamageDistanceOffset,maxLiquidFuel,maxOxidizer,maxOre,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,variant_y_offsets,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider
str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,str,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""24-77 ""Twitch""…",0.08,230.0,7.0,2000,"""precisionPropu…","""Rockomax Congl…","""Tiny engine! B…","""RoverDude""","""Part""",1.0,0.0,16.0,920,"[0, 1, … 0]",0.06,4.0,0.8,"""default""",0.2,0.2,2.0,"[""srf""]","""Gimbal""",8.0,30.0,true,null,null,null,null,null,null,65,null,null,null,…,150.0,0.12,null,null,null,null,null,null,null,null,null,null,null,null,"[""LiquidFuel"", ""Oxidizer""]",null,null,null,275.0,290.0,"""orange propuls…","""Squad/Parts/En…","""Engine""","""0""","""/home/zeke/Sof…","""smallRadialEng…",200.0,null,0.08,200.0,3625.0,3625.0,3437.5,3437.5,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad"""
"""EM-64 Heavy Ro…",0.6,600.0,12.0,2000,"""advancedMotors…","""Kerbal Motion …","""A large low-to…","""RoverDude""","""Part""",1.0,null,null,8200,"[1, 1, … 0]",null,null,null,"""default""",0.2,0.2,1.0,"[""size1""]",null,null,null,null,null,null,null,null,null,null,-1,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, 1.25, … 2.0]","[0.0, 0.0, … 2.0]",null,null,null,"""robot actuator…","""SquadExpansion…","""Robotics""","""0""","""/home/zeke/Sof…","""rotor_03""",null,1.25,0.6,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""SquadExpansion…"
"""RC-15 RCS Thru…",0.0125,20.0,15.0,1500,"""advFlightContr…","""STEADLER Engin…","""Miniaturizing …","""Chris Adderley…","""Part""",null,null,null,0,"[0, 1, … 0]",null,null,null,"""default""",0.001,0.001,2.0,"[""srf""]",null,null,null,null,null,null,null,null,null,null,8,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""restock cluste…","""ReStockPlus/As…","""none""",null,"""/home/zeke/Sof…","""restock-rcs-bl…",null,null,0.0125,null,null,null,null,null,null,null,"""ReStockPlus"""
"""Engine Nacelle…",0.15,600.0,10.0,2000,"""aerodynamicSys…","""C7 Aerospace D…","""The Engine Nac…","""C. Jenkins, Po…","""Part""",1.0,null,null,6500,"[1, 1, … 0]",null,null,0.6,"""default""",0.2,0.3,1.0,"[""size1"", ""srf""]",null,null,null,null,null,null,null,null,null,null,-1,null,null,null,…,null,null,150.0,null,null,null,5.0,null,150.0,null,null,null,5.0,null,null,"[0.0, 0.9375, … 0.0]","[0.0, -0.9375, … 0.0]",null,null,null,"""aero (air airc…","""Squad/Parts/St…","""Aero""","""0""","""/home/zeke/Sof…","""nacelleBody""",null,1.875,0.9,null,null,null,null,null,null,null,"""Squad"""
"""RC-001S Remote…",0.1,2250.0,9.0,2000,"""advUnmanned""","""STEADLER Engin…","""The STEADLER R…","""Squad""","""Part""",1.0,null,null,17800,"[1, 0, … 0]",null,null,null,"""default""",0.2,0.2,2.0,"[""size1""]",null,null,null,null,null,null,null,null,null,null,400,null,null,null,…,null,null,nul

In [10]:
booster_parts = create_booster(df.filter(c('provider').str.contains('Squad')), 100, 'size4')

In [11]:
copy(compile_parts(booster_parts, 'NAME_OF_SAVE', 'fairing_12345', 12))

TypeError: compile_parts() takes 3 positional arguments but 4 were given

In [11]:
save = read_existing_save(df, '/home/zeke/Software/KSP_linux/saves/TESTING_ONLY/Ships/VAB/TEST_PARTS.craft')

In [38]:
save = convert_list_to_dict(parse_cfg_file('/home/zeke/Software/KSP_linux/saves/TESTING_ONLY/Ships/VAB/TEST_PARTS.craft').as_list())

In [12]:
test_save = '/home/zeke/Software/KSP_linux/saves/TESTING_ONLY/Ships/VAB/TEST_PARTS.craft'

# TODO: part names must be unique for read_existing_save() to work

In [409]:
save_file = Path(test_save)
str(save_file.parent / save_file.name[:-6]) + '_booster.craft'

'/home/zeke/Software/KSP_linux/saves/TESTING_ONLY/Ships/VAB/TEST_PARTS_booster.craft'

In [388]:
df.filter(c('name') == 'restock-fairing-base-0625-1')

title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,experimentActionName,…,heatProduction,exhaustDamageDistanceOffset,maxLiquidFuel,maxOxidizer,maxOre,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,variant_y_offsets,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider
str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,str,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""AE-FF0 Airstre…",0.01875,150.0,9.0,2600,"""advConstructio…","""FLOOYD Dynamic…","""While the Kerb…","""Chris Adderley…","""Part""",1.0,null,null,1500,"[1, 0, … 0]",null,0.25,0.8,"""default""",0.2,0.2,2.0,"[""size0""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.138, … 0.0]","[0.0, -0.045, … 0.0]",null,null,null,"""restock aero )…",null,"""Payload""",null,"""/home/zeke/Sof…","""restock-fairin…",null,0.183,0.01875,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""ReStockPlus"""


In [410]:
# generate_booster_save(df, test_save)
# generate_booster_save(df, '/home/zeke/Software/KSP_linux/saves/TESTING_ONLY/Ships/VAB/TEST PARTS New.craft')
generate_booster_save(df, '/home/zeke/Software/KSP_linux/saves/TESTING_ONLY/Ships/VAB/Actual Test.craft')

Generating Booster for payload named "Actual Test"
Payload has a mass of 0.991T, and a size1p5 fairing at [0.0, 12.495403097, 0.0]
Calculated deltaV: 1160.77931305366 for diameter size1p5
Calculated deltaV: 3328.7043219176776 for diameter size2


id,title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,…,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,variant_y_offsets,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider,stage,part_id,prev_part_id,next_part_id,height
u32,str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,i32,str,str,str,f64
1,"""TD-12 Decouple…",0.04,200.0,6.0,2000,"""engineering101…","""O.M.B. Demolit…","""The TD-12 Stac…","""RoverDude""","""Part""",1.0,null,null,1600,"[1, 0, … 0]",null,null,null,"""default""",0.2,0.2,2.0,"[""size1""]",null,null,null,null,null,null,null,null,null,null,260,null,null,…,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.05, … 1.0]","[0.0, -0.05, … 1.0]",null,null,null,"""decouple""","""Squad/Parts/Co…","""Coupling""","""0""","""/home/zeke/Sof…","""Decoupler_1""",null,0.1,0.04,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad""",-1,"""Decoupler.1_1""",null,"""Size1p5.Tank.0…",12.445403
2,"""FL-TX220 Fuel …",0.1375,220.0,6.0,2000,"""advRocketry""","""Jebediah Kerma…","""Designed as an…","""RoverDude""","""Part""",1.0,null,null,1750,"[1, 1, … 0]",null,null,null,"""default""",0.2,0.3,2.0,"[""size1p5"", ""srf""]",null,null,null,null,null,null,null,null,null,null,-1,null,null,…,null,null,null,99.0,121.0,null,null,null,null,null,"[0.0, 0.234375, … 2.0]","[0.0, -0.234375, … 2.0]",null,null,null,"""rocket fuel li…","""SquadExpansion…","""FuelTank""","""0""","""/home/zeke/Sof…","""Size1p5_Tank_0…",null,0.46875,1.2375,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""SquadExpansion…",0,"""Size1p5.Tank.0…","""Decoupler.1_1""","""LiquidEngineLV…",12.161028
3,"""LV-T91 ""Cheeta…",1.0,850.0,7.0,2000,"""heavierRocketr…","""Jebediah Kerma…","""This engine wi…","""RoverDude""","""Part""",1.0,0.0,125.0,3400,"[1, 1, … 0]",null,null,null,"""default""",0.2,0.3,2.0,"[""size1p5"", ""srf""]","""gimbal""",4.0,20.0,true,"""Shroud1p5x2,Sh…","""bottom""",true,0.1,5.0,"[0, 0, 1]",-1,5.0,null,…,null,null,null,null,null,null,null,null,null,"[""LiquidFuel"", ""Oxidizer""]","[0.0, 0.84028, … 2.0]","[0.0, -1.2, … 2.0]","[-1.635, -1.66]",150.0,355.0,"""rocket liquid …",null,"""Engine""","""0""","""/home/zeke/Sof…","""LiquidEngineLV…",125.0,2.47528,1.0,125.0,355.0,355.0,150.0,150.0,"""/home/zeke/Sof…","""http://0.0.0.0…","""SquadExpansion…",0,"""LiquidEngineLV…","""Size1p5.Tank.0…","""Decoupler.1_4""",11.086373
4,"""TD-12 Decouple…",0.04,200.0,6.0,2000,"""engineering101…","""O.M.B. Demolit…","""The TD-12 Stac…","""RoverDude""","""Part""",1.0,null,null,1600,"[1, 0, … 0]",null,null,null,"""default""",0.2,0.2,2.0,"[""size1""]",null,null,null,null,null,null,null,null,null,null,260,null,null,…,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.05, … 1.0]","[0.0, -0.05, … 1.0]",null,null,null,"""decouple""","""Squad/Parts/Co…","""Coupling""","""0""","""/home/zeke/Sof…","""Decoupler_1""",null,0.1,0.04,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad""",0,"""Decoupler.1_4""","""LiquidEngineLV…","""Size1p5.Size2.…",9.401373
5,"""

['Decoupler_1',
 'Size1p5_Tank_01',
 'LiquidEngineLV-T91',
 'Decoupler_1',
 'Size1p5_Size2_Adapter_01',
 'Rockomax8BW',
 'Rockomax8BW',
 'Rockomax8BW',
 'Rockomax8BW',
 'Rockomax8BW',
 'Rockomax8BW',
 'Rockomax8BW',
 'Rockomax8BW',
 'Rockomax8BW',
 'SSME']

['TD-12 Decoupler',
 'FL-TX220 Fuel Tank',
 'LV-T91 "Cheetah" Liquid Fuel Engine',
 'TD-12 Decoupler',
 'FL-A215 Fuel Tank Adapter',
 'Rockomax X200-8 Fuel Tank',
 'Rockomax X200-8 Fuel Tank',
 'Rockomax X200-8 Fuel Tank',
 'Rockomax X200-8 Fuel Tank',
 'Rockomax X200-8 Fuel Tank',
 'Rockomax X200-8 Fuel Tank',
 'Rockomax X200-8 Fuel Tank',
 'Rockomax X200-8 Fuel Tank',
 'Rockomax X200-8 Fuel Tank',
 'S3 KS-25 "Vector" Liquid Fuel Engine']

[12.445403097,
 12.161028096999999,
 11.086373097,
 9.401373096999999,
 8.413873096999998,
 7.021373096999998,
 6.111373096999998,
 5.201373096999998,
 4.2913730969999975,
 3.3813730969999973,
 2.471373096999997,
 1.561373096999997,
 0.6513730969999969,
 -0.2586269030000031,
 -0.7136269030000031]

In [390]:
df.filter(c('title') == 'RV-1 "Cub" Vernier Engine')

title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,experimentActionName,…,heatProduction,exhaustDamageDistanceOffset,maxLiquidFuel,maxOxidizer,maxOre,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,variant_y_offsets,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider
str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,str,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""RV-1 ""Cub"" Ver…",0.18,800.0,7.0,2000,"""precisionPropu…","""Reaction Syste…","""Mew! These lit…","""RoverDude""","""Part""",1.0,0.0,32.0,3200,"[0, 1, … 0]",null,null,null,"""default""",0.2,0.3,2.0,"[""size0"", ""srf""]","""Gimbal""",22.5,null,true,null,null,null,null,null,null,170,null,null,null,…,150.0,0.15,null,null,null,null,null,null,null,null,null,null,null,null,"[""LiquidFuel"", ""Oxidizer""]",null,null,null,280.0,310.0,"""rocket liquid …","""SquadExpansion…","""Engine""","""0""","""/home/zeke/Sof…","""LiquidEngineRV…",177.777778,null,0.18,177.777778,1722.222222,1722.222222,1555.555556,1555.555556,"""/home/zeke/Sof…","""http://0.0.0.0…","""SquadExpansion…"


In [315]:
len(df['name']) - len(set(df['name']))

24

In [317]:
len(df) -len(df.group_by('name').agg(pl.first()))

24

In [257]:
stack = df.filter((~pl.col('node_stack_top').is_null()) & (~pl.col('node_stack_bottom').is_null())).sample(10)

In [261]:
y = 100
heights = []
for i in stack.iter_rows(named=True):
    y += i['node_stack_top'][1]
    heights.append(y)
    y += i['node_stack_bottom'][1]
heights

[100.1108553,
 101.049,
 100.8,
 98.213,
 98.493,
 99.705,
 99.46699999999998,
 98.437,
 98.837,
 99.4136]

In [236]:
a = np.array((0.0416163057,14.079161097,-0.15590176))
b = np.array((0.0416163057,14.4416609,-0.15590176))
c = np.array((0.0416163057,14.5666609,-0.15590176))
print(a - b)
print(a - c)

[ 0.        -0.3624998  0.       ]
[ 0.        -0.4874998  0.       ]


In [53]:
parts = df
parts.filter(c('provider').str.contains('Squad')).filter(
            c('diameter').list.contains('size4') &
            (c('category') == 'FuelTank') &
            (c('maxLiquidFuel') > 0) &
            (c('maxOxidizer') > 0) &
            (~c('title').is_in(("R-12 'Doughnut' External Tank",))) &
            (~c('title').str.contains('(?i)adapter'))
        ).sort('wet_mass')[0]

title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,experimentActionName,…,thrustVectorTransformName,heatProduction,exhaustDamageDistanceOffset,maxLiquidFuel,maxOxidizer,maxOre,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider
str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,str,…,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""Kerbodyne S4-6…",4.0,6400.0,10.0,2000,"""highPerformanc…","""Kerbodyne""","""Originally the…","""Pablo""","""Part""",1.0,null,null,25600,"[1, 1, … 0]",null,null,null,"""default""",0.2,0.3,2.0,"[""size4"", ""srf""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,2880.0,3520.0,null,null,null,null,2880.0,3520.0,null,null,null,null,null,"[0.0, 0.9375, … 4.0]","[0.0, -0.9375, … 4.0]",null,null,"""rocket fuel li…","""SquadExpansion…","""FuelTank""","""0""","""/home/zeke/Sof…","""Size4_Tank_01""",null,1.875,36.0,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""SquadExpansion…"


In [46]:
decoupler_diameter = 'size1'
df.filter(
    (c('diameter').list.contains(decoupler_diameter)) &
    (c('category') == 'Coupling') &
    (c('title').str.contains('(?i)decouple')) &
    (c('provider') == 'Squad')
)

title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,experimentActionName,…,thrustVectorTransformName,heatProduction,exhaustDamageDistanceOffset,maxLiquidFuel,maxOxidizer,maxOre,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider
str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,str,…,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""TD-12 Decouple…",0.04,200.0,6.0,2000,"""engineering101…","""O.M.B. Demolit…","""The TD-12 Stac…","""RoverDude""","""Part""",1.0,null,null,1600,"[1, 0, … 0]",null,null,null,"""default""",0.2,0.2,2.0,"[""size1""]",null,null,null,null,null,null,null,null,null,null,260,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.05, … 1.0]","[0.0, -0.05, … 1.0]",null,null,"""decouple""","""Squad/Parts/Co…","""Coupling""","""0""","""/home/zeke/Sof…","""Decoupler_1""",null,0.1,0.04,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad"""


In [120]:
with open(test_save) as f:
    test_contents = f.read()

In [ ]:
print(test_contents)

In [ ]:
fairing_id = 'fairingSize1_4294172644'
THIS = '!!!!!!!!!!!!!!!!!!!!!!!!!!!'
print(re.sub(fr'(?<=part\ =\ {fairing_id})(?P<n>(?:(?:.|\n))+attN\ =\ bottom,)(?P<o>.+)(?=\|.+)', fr'\g<n>{THIS}', test_contents))

In [106]:
df

title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,experimentActionName,…,thrustVectorTransformName,heatProduction,exhaustDamageDistanceOffset,maxLiquidFuel,maxOxidizer,maxOre,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider
str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,str,…,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""FAT-455 Aeropl…",0.17,800.0,15.0,1200,"""heavyAerodynam…","""WinterOwl Airc…","""Large conventi…","""Porkjet""","""Part""",1.0,null,null,16400,"[0, 1, … 1]",0.12,null,0.4,"""none""",0.02,0.02,2.0,"[""srf""]",null,null,null,null,null,null,null,null,null,null,-1,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""aileron aircra…","""Squad/Parts/Ae…","""Aero""","""0""","""/home/zeke/Sof…","""airlinerCtrlSr…",null,null,0.17,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad"""
"""Atmospheric Fl…",0.005,6500.0,8.0,1200,"""scienceTech""","""Probodobodyne …","""A device for p…","""NovaSilisko""","""Part""",0.8,null,null,12200,"[0, 1, … 1]",null,null,null,"""default""",0.2,0.2,1.0,"[""srf""]",null,null,null,null,null,null,null,null,null,null,130,null,"""atmosphereAnal…","""Run Atmosphere…",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""atmospher expe…",null,"""Science""","""0""","""/home/zeke/Sof…","""sensorAtmosphe…",null,null,0.005,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad"""
"""Snacks!""",0.001,6.0,6.0,2200,"""advExploration…","""Kerbal Invento…","""To be used in …","""Well""","""Part""",null,null,null,15,"[0, 0, … 0]",null,null,null,"""default""",0.2,0.3,2.0,"[""srf""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""cck-eva-items""",null,"""none""","""0""","""/home/zeke/Sof…","""KIS_snacks""",null,null,0.001,null,null,null,null,null,null,null,"""KIS"""
"""Communotron 88…",0.1,1500.0,7.0,2000,"""automation""","""Ionic Symphoni…","""The Communotro…","""NovaSilisko""","""Part""",1.0,null,null,20000,"[1, 1, … 0]",null,null,null,"""default""",0.2,0.2,2.0,"[""size0"", ""srf""]",null,null,null,null,null,null,null,null,null,null,60,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, -0.13, … 0.0]",null,null,"""aerial antenna…",null,"""Communication""","""0""","""/home/zeke/Sof…","""commDish""",null,null,0.1,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad"""
"""EM-64S Heavy R…",0.55,600.0,12.0,2000,"""advancedMotors…","""Kerbal Motion …","""A large low-to…","""RoverDude""","""Part""",1.0,null,null,8200,"[1, 1, … 0]",null,null,null,"""default""",0.2,0.2,1.0,"[""size1""]",null,null,null,null,null,null,null,null,null,null,-1,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.955512, … 1.0]","[0.0, 0.0, …

In [89]:
save = convert_list_to_dict(parse_cfg_file(test_save).as_list())
matches = [m.value for m in parse('$.PART..part').find(save)]
largest_fairing = df.filter(c('name').is_in([m.split('_')[0].replace('.', '_') for m in matches if 'fairing' in m])).sort('dry_mass')[-1]
largest_fairing_id = list(filter(lambda i: largest_fairing['name'][0] in i, matches))[0]

In [12]:
df.filter(
            (c('diameter').list.contains('size1')) &
            (c('category') == 'Coupling') &
            (c('title').str.contains('(?i)decouple')) &
            (c('provider') == 'Squad')
        )

title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,experimentActionName,…,thrustVectorTransformName,heatProduction,exhaustDamageDistanceOffset,maxLiquidFuel,maxOxidizer,maxOre,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider
str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,str,…,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""TD-12 Decouple…",0.04,200.0,6.0,2000,"""engineering101…","""O.M.B. Demolit…","""The TD-12 Stac…","""RoverDude""","""Part""",1.0,null,null,1600,"[1, 0, … 0]",null,null,null,"""default""",0.2,0.2,2.0,"[""size1""]",null,null,null,null,null,null,null,null,null,null,260,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.05, … 1.0]","[0.0, -0.05, … 1.0]",null,null,"""decouple""","""Squad/Parts/Co…","""Coupling""","""0""","""/home/zeke/Sof…","""Decoupler_1""",null,0.1,0.04,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad"""


In [13]:
df.filter(c('provider').str.contains('Squad')).filter(
    c('diameter').list.contains('size4') &
    (c('category') == 'FuelTank') &
    (c('maxLiquidFuel') > 0) &
    (c('maxOxidizer') > 0) &
    (~c('title').is_in(("R-12 'Doughnut' External Tank",))) &
    (~c('title').str.contains('(?i)adapter'))
).sort('wet_mass')['title'][0]

'Kerbodyne S4-64 Fuel Tank'

In [14]:
i = 0
print(optimal_engine(df.filter(c('provider').str.contains('Squad')), 1, 9, 'size3', True, 1.5)['title'][i])
print('TWR:', optimal_engine(df.filter(c('provider').str.contains('Squad')), 1, 9, 'size3', True, 1.5).select((c('maxThrust')/(9 + c('wet_mass'))/10))[i,0])
optimal_engine(df.filter(c('provider').str.contains('Squad')), 1, 9, 'size3', True, 1.5)

RE-J10 "Wolfhound" Liquid Fuel Engine
TWR: 3.0487804878048776


title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,experimentActionName,…,thrustVectorTransformName,heatProduction,exhaustDamageDistanceOffset,maxLiquidFuel,maxOxidizer,maxOre,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider
str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,str,…,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""RE-J10 ""Wolfho…",3.3,3000.0,6.0,2000,"""veryHeavyRocke…","""Rockomax Congl…","""Woof! With the…","""RoverDude""","""Part""",1.0,0.0,375.0,12000,"[1, 0, … 0]",0.06,4.0,0.8,"""default""",0.2,0.2,2.0,"[""size2""]","""gimbal""",3.0,null,null,"""Shroud2x2,Shro…","""bottom""",true,0.1,5.0,"[0, 0, 1]",null,8.0,null,null,…,"""thrustTransfor…",83.0,1.75,null,null,null,null,null,null,null,null,null,null,null,null,"[""LiquidFuel"", ""Oxidizer""]","[0.0, 0.361067, … 2.0]","[0.0, -1.8, … 2.0]",70.0,380.0,"""rocket liquid …",null,"""Engine""","""0""","""/home/zeke/Sof…","""LiquidEngineRE…",113.636364,2.161067,3.3,113.636364,115.151515,115.151515,21.212121,21.212121,"""/home/zeke/Sof…","""http://0.0.0.0…","""SquadExpansion…"


In [15]:
# len(df)
len(df) - len(df['name'].unique())
a = df.filter(df['name'].is_duplicated())
a

title,dry_mass,cost,crashTolerance,maxTemp,techLevel,manufacturer,description,author,module,rescaleFactor,minThrust,maxThrust,entryCost,attachRules,heatConductivity,skinInternalConductionMult,emissiveConstant,dragModelType,maximumDrag,minimumDrag,angularDrag,diameter,gimbalTransformName,gimbalRange,gimbalResponseSpeed,useGimbalResponseSpeed,jettisonName,bottomNodeName,isFairing,jettisonedObjectMass,jettisonForce,jettisonDirection,packedVolume,alternatorRate,experimentID,experimentActionName,…,thrustVectorTransformName,heatProduction,exhaustDamageDistanceOffset,maxLiquidFuel,maxOxidizer,maxOre,maxElectricCharge,maxIntakeAir,maxMonoPropellant,defaultLiquidFuel,defaultOxidizer,defaultOre,defaultElectricCharge,defaultIntakeAir,defaultMonoPropellant,propellants,node_stack_top,node_stack_bottom,ISP_atm,ISP_vac,tags,model_path,category,subcategory,path,name,TWR_dry,est_height,wet_mass,TWR_wet,ISP_vac_WR_dry,ISP_vac_WR_wet,ISP_atm_WR_dry,ISP_atm_WR_wet,icon_path,icon_url,provider
str,f64,f64,f64,i64,str,str,str,str,str,f64,f64,f64,i64,list[i64],f64,f64,f64,str,f64,f64,f64,list[str],str,f64,f64,bool,str,str,bool,f64,f64,list[i64],i64,f64,str,str,…,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[str],list[f64],list[f64],f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""M-Beam 200 I-B…",0.1875,14.0,80.0,2000,"""composites""","""Dinkelstein Ke…","""Handcrafted by…","""NovaSilisko""","""Part""",1.0,null,null,6200,"[1, 1, … 1]",null,null,null,"""default""",0.2,0.2,1.0,"[""size0"", ""srf""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.8024656, … 0.0]","[0.0, -0.802466, … 0.0]",null,null,"""beam build con…",null,"""Structural""","""0""","""/home/zeke/Sof…","""structuralIBea…",null,1.6049312,0.1875,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad"""
"""HG-5 High Gain…",0.07,600.0,8.0,2000,"""basicScience""","""Ionic Symphoni…","""A short range …","""Squad""","""Part""",1.0,null,null,1500,"[1, 1, … 0]",null,null,null,"""default""",0.2,0.2,1.0,"[""srf""]",null,null,null,null,null,null,null,null,null,null,60,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.0, … 0.0]",null,null,null,"""Squad/Parts/Ut…","""Communication""","""0""","""/home/zeke/Sof…","""HighGainAntenn…",null,null,0.07,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad"""
"""Inflatable Air…",0.1,400.0,10.0,2600,"""advMetalworks""","""Jebediah Kerma…","""When trying to…","""RoverDude""","""Part""",1.0,null,null,8400,"[1, 1, … 0]",null,null,0.8,"""default""",0.25,0.25,0.5,"[""size0""]",null,null,null,null,null,null,null,null,null,null,520,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.0, … 1.0]",null,null,"""connect join d…","""SquadExpansion…","""Coupling""","""0""","""/home/zeke/Sof…","""InflatableAirl…",null,null,0.1,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""SquadExpansion…"
"""Probodobodyne …",0.07,360.0,15.0,1200,"""unmannedTech""","""Probodobodyne …","""QBE is Probodo…","""AlexanderM""","""Part""",1.0,null,null,3800,"[1, 0, … 0]",null,null,null,"""default""",0.2,0.15,1.5,"[""size0""]",null,null,null,null,null,null,null,null,null,null,140,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[0.0, 0.2845967, … 0.0]","[0.0, -0.284597, … 0.0]",null,null,"""command contro…",null,"""Pods""","""0""","""/home/zeke/Sof…","""probeCoreCube""",null,0.5691934,0.07,null,null,null,null,null,"""/home/zeke/Sof…","""http://0.0.0.0…","""Squad"""
"""RA-2 Relay Ant…",0.15,1800.0,8.0,2000,"""precisionEngin…","""Ionic Symphoni…","""A relay antenn…","""Squad""","""Part""",1.0,null,null,5000,"[1, 1, … 0]",null,null,null,"""default""",0.2,0.2,1.0,"[""size0""]",null,null,null,null,null,null,null,null,null

In [16]:
# df.filter(~c('propellants').is_null())['propellants']
df.filter(c('diameter').list.contains('size1'))['diameter']

diameter
list[str]
"[""size1""]"
"[""size1""]"
"[""size1"", ""srf""]"
"[""size1""]"
"[""size1"", ""srf""]"
…
"[""size1"", ""size0""]"
"[""size1"", ""srf""]"
"[""size2"", ""size1p5"", … ""srf""]"


In [17]:
df['category'].unique().to_list()

['Payload',
 'Propulsion',
 'Coupling',
 'Science',
 'Aero',
 'Utility',
 '-1',
 'Ground',
 'Structural',
 'Communication',
 'Engine',
 'FuelTank',
 'Thermal',
 'none',
 'Cargo',
 'Pods',
 'Control',
 'Electrical',
 'Robotics']

In [40]:
from clipboard import copy
# name = 'Kerbodyne S3-7200 Tank'
name = 'fuelTankSmallFlat'
copy(df.filter(c('name') == name)['title'][0])

In [103]:
16.013066097-0.3125

15.700566097

In [ ]:
# Testing the parser
# For testing removing requirements
p = '/home/zeke/Software/KSP_linux/GameData/ExtraplanetaryLaunchpads/Parts/Auger/part.cfg'
# TMP: I have *no idea* why this *one* part won't load properly
# p = '/home/zeke/Software/KSP_linux/GameData/Squad/Parts/Engine/liquidEngineMainsail_v2/liquidEngineMainsail_v2.cfg'
convert_list_to_dict(parse_cfg_file(p, True).as_list())

In [279]:
# Testing JSONPaths
# [m.value for m in parse('$[?(@.name==turboJet)].MODULE[?(@.name==ModuleEnginesFX)].atmosphereCurve.key').find(all_parts)]
# part_name = 'batteryBankMini'
part_name = 'liquidEngine3_v2'
# path = 'MODULE[?(@.name==ModuleEnginesFX)].PROPELLANT..name'
# path = '.MODULE[?(@.name==ModulePartVariants)]..VARIANT[0]..NODES.node_stack_bottom.`split(", ", 1, -1)`'
path = '.dstg'
# path = 'node_stack_top.`split(1, 0, -1)`'
# path = 'node_stack_top.`split(", ", 1, -1)`'
# path = 'RESOURCE'
[m.value for m in parse(f'$[?(@.name=={part_name})]' + path).find(all_parts)]
# parse('$[?(@.name==liquidEngine3_v2)].MODULE[?(@.name==ModuleEngines)].atmosphereCurve').find(all_parts)

[]

In [97]:
[
    m.value for m in parse(
        '$[?(@.name==liquidEngine3_v2)].atmosphereCurve'
    ).find(all_parts)
]

[]

In [22]:
parse(f'$[?(@.name=={part_name})].' + path)
from jsonpath_ng.ext.string import Split


- LiquidFuel: 0.005 Tons / unit
- Oxidizer: 0.005 Tons / unit
- Ore: 0.01 Tons / unit
- MonoProp: 0.004 Tons / unit